In [1]:
import os
import random
import argparse
import numpy as np

import torch

from models.networks.ofa_resnets import OFAResNets
from datasets import load_dataset, load_from_disk


In [2]:
# set random seed
random_seed = 1
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

In [3]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda_available = torch.cuda.is_available()
if cuda_available:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(random_seed)
    print('Using GPU.')
else:
    print('Using CPU.')

Using GPU.


In [4]:
parser = argparse.ArgumentParser()

parser.add_argument("--n_classes", type=int, default=1000)  # imagenet:1000 / tinyimagenet:200 / casiaweb: ?


args = parser.parse_args(args=[])

args.ks_list = "3,5,7"
args.depth_list = "3,4"
args.expand_list = "1"
args.width_mult_list = "1.0"

In [5]:
# build net from args
args.width_mult_list = [
    float(width_mult) for width_mult in args.width_mult_list.split(",")
]
args.width_mult_list = (
    args.width_mult_list[0]
    if len(args.width_mult_list) == 1
    else args.width_mult_list
)
args.ks_list = [int(ks) for ks in args.ks_list.split(",")]
args.expand_list = [int(e) for e in args.expand_list.split(",")]
args.depth_list = [int(d) for d in args.depth_list.split(",")]

net = OFAResNets(
    n_classes = args.n_classes, 
    depth_list = args.depth_list,
    expand_ratio_list = args.expand_list,
    width_mult_list = args.width_mult_list
)

# pretrained
init = torch.load(
    f="exp/kernel2kernel_depth/phase1/checkpoint/tinyImageNet/model_best.pth.tar",
    # map_location="cpu"  # if you want to load cpu add map_location="cpu"
)["state_dict"]

net.load_state_dict(init)

ofa_network = net
print('The OFA Network is ready.')

The OFA Network is ready.


In [6]:
from huggingface_hub import login
login()

In [7]:
imagenet_data_path = 'dataset/imagenet'
cache_dir = "/data"

# ImageNet-1k 데이터셋 로드
dataset = load_dataset("imagenet-1k", cache_dir=cache_dir)

# Train, Validation, Test 세트 분리
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]
test_dataset = dataset["test"]

# 각 세트를 지정된 경로에 저장
train_dataset.save_to_disk(f"{imagenet_data_path}/train")
validation_dataset.save_to_disk(f"{imagenet_data_path}/val")
test_dataset.save_to_disk(f"{imagenet_data_path}/test")

print('The ImageNet dataset files are ready.')

/home/heonsung/anaconda3/envs/lab/lib/python3.8/site-packages/datasets/load.py:1429: FutureWarning: The repository for imagenet-1k contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/imagenet-1k
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


PermissionError: [Errno 13] Permission denied: '/data/_data_imagenet-1k_default_1.0.0_09dbb3153f1ac686bac1f40d24f307c383b383bc171f2df5d9e91c1ad57455b9.lock'